# README
Here we tried to create two scores functions:
1. score each speaker on his participation on each session  

In [1]:
import numpy as np
import sys, os
import pandas as pd
import re
import nltk
import warnings
import pickle
import gensim
import plotly.offline as py
import plotly.graph_objs as go

PROJECT_ROOT = r'C:\Users\lotem\knesset_data'
META_SOURCE = os.path.join(PROJECT_ROOT, 'data_committees_kns_committeesession_kns_committeesession.csv')
COMMITTEESS_ROOT = os.path.join('D:\knesset', 'speakers')
PEOPLE_SOURCE = os.path.join(PROJECT_ROOT, 'data_members_kns_person_kns_person.csv')
WORD_COUNT_TARGET = os.path.join(PROJECT_ROOT, 'word_count')

meta = pd.read_csv(META_SOURCE)
meta = meta[~meta['parts_parsed_filename'].isnull()]

In [11]:
def segment(session_df):
    session_df['len_body'] = (session_df.body.astype(str)).apply(lambda body: len(body.split(" ")))
    num_segments = 6
    session_len = len(session_df)
    floored_session_len = session_len - (session_len % 6)
    segment_len = round(floored_session_len/num_segments)
    border_list = list(range(0, session_len, segment_len)) 
    if (len(border_list) == 6):
        border_list.append(session_len)
    else:
        border_list[-1] = session_len
    segments_dfs = [session_df.iloc[border_list[i]:border_list[i+1],:] for i in range(len(border_list) - 1)]
    assert len(segments_dfs) == num_segments
    return segments_dfs

In [12]:
def score_pm_on_segment(pm_id, df, shouts_th = 10, percentage_th = 0.1):
    meaningfull_content = (df.len_body[df.len_body > shouts_th]).sum()
    pm_meaningfull_content = (df[(df.len_body > shouts_th) & (df.PersonID == pm_id)]).len_body.sum()
    per = pm_meaningfull_content/meaningfull_content
    if (per > percentage_th):
        return 1
    return 0

In [15]:
def get_scores(session_df):
    segments_dfs = segment(session_df)
    ids = list(set(list((session_df.PersonID).dropna())))
    scores = [sum([score_pm_on_segment(id, seg) for seg in segments_dfs])/6.0 for id in ids]
    df = pd.DataFrame({'PersonID': ids, 'Score' : scores})
    return df

<u> Iterate over all rows in the meta files and calculate the score for each session for each speaker </u>
The df - pandas.DataFrame - is the main output of this stage

In [24]:
def get_full_url(metadf, root_folder):
    if type(metadf) is pd.DataFrame:
        return [os.path.join(root_folder,'speakers', x[1]['parts_parsed_filename']) 
            for x in metadf.iterrows()]
    else:
        return os.path.join(root_folder,'speakers', metadf['parts_parsed_filename']) 

subdfs = []
for meta_row in meta.iterrows():
#     try:
    subdf = get_scores(pd.read_csv(get_full_url(meta_row[1], root_folder='D:\knesset')))
    subdf['CommitteeSessionID'] = meta_row[1]['CommitteeSessionID']
    subdfs.append(subdf)
#     except:
#         s = meta_row[1]['CommitteeSessionID']
#         warnings.warn(f'error with {s}')
df = pd.concat(subdfs, axis=0, ignore_index=True)
df.columns = ['PersonID', 'score','CommitteeSessionID']
df

,PersonID,score,CommitteeSessionID
0,1032.0,0.000000,64550
1,1033.0,0.166667,64550
2,1068.0,0.000000,64550
3,23596.0,0.333333,64550
4,430.0,0.500000,64550
5,1041.0,0.000000,64550
6,1046.0,0.333333,64550
7,1049.0,0.000000,64550
8,1068.0,0.000000,64551
9,1036.0,0.000000,64551


<b> here we didn't finish the job, we were suppose to combine the table from the object "id_to_lemma_LDA_100_1p0.pkl"
    this object have the score for each committee session to each topic, we should merge this score with the score of each subject to each session to create a score of each subject to each topic </b>

In [26]:
id2topicscore = pickle.load(open(r"C:\Users\lotem\knesset_data\id_to_lemma_LDA_100_1p0.pkl",'rb'))
id2topicscore

,CommitteeSessionID,lemmas,topics
0,64990,"{'JJ': [], 'CD': ['52'], 'NN': ['בחירה', 'מס']...","[(52, 0.402), (72, 0.402)]"
1,470756,"{'JJ': ['תורכי'], 'CD': [], 'NN': ['מדינה', 'ה...","[(5, 0.335), (81, 0.16833335), (97, 0.335)]"
2,470814,"{'JJ': [], 'CD': [], 'NN': ['ציבור', 'תוכנית',...","[(20, 0.12625), (34, 0.12625), (59, 0.37625), ..."
3,471255,"{'JJ': ['בריא', 'נאות'], 'CD': ['תשע', '2012',...","[(6, 0.4369498), (10, 0.063125), (49, 0.250625..."
4,471661,"{'JJ': ['ביולוגי'], 'CD': [], 'NN': ['מעמד', '...","[(12, 0.14428571), (15, 0.14428571), (18, 0.14..."
5,471662,"{'JJ': [], 'CD': [], 'NN': ['מעמד', 'עובד', 'ה...","[(16, 0.12625), (18, 0.12625), (31, 0.12625), ..."
6,472322,"{'JJ': [], 'CD': [], 'NN': ['ישיבה', 'קליטה', ...","[(19, 0.091818176), (25, 0.45545456), (27, 0.1..."
7,474249,"{'JJ': [], 'CD': [], 'NN': ['ישיבה', 'עלייה'],...","[(15, 0.12625), (22, 0.12625001), (25, 0.50124..."
8,64515,"{'JJ': [], 'CD': [], 'NN': [], 'NNT': [], 'NNP...",[]
9,64550,"{'JJ': [], 'CD': [], 'NN': ['תעופה'], 'NNT': [...","[(30, 0.16833334), (59, 0.66833335)]"
